In [1]:
!pip install accelerate
!pip install langchain-community
!pip install langchain
!pip install sentence-transformers
!pip install pdfplumber
!pip install chromadb
!pip install tiktoken
! pip install transformers trl py7zr optimum accelerate bitsandbytes
! pip install git+https://github.com/huggingface/peft
! pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118
!pip install transformers accelerate bitsandbytes -q
!pip install -U bitsandbytes
!pip install wikipedia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 7.9 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5

In [26]:
!pip install youtube-search-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.5/99.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 6.0 MB/s eta 0:00:00


In [34]:
!pip install youtube-search

In [30]:
from openai import OpenAI
import re
from langchain_core.runnables import Runnable
from langchain.agents import AgentType, initialize_agent, load_tools, Tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.youtube.search import YouTubeSearchTool
from langchain.chains import LLMMathChain

In [ ]:
client = OpenAI(base_url='https://api.gapapi.com/v1', api_key='sk-9AtuC5pfE0lAZ2DEGdEzRsEgPXugkaJxBNefdBCSSXgE3G1S')

In [4]:
class SimpleLLM(Runnable):
    def invoke(self, prompt, *args, **kwargs) -> str:
        if not isinstance(prompt, str):
            if isinstance(prompt, dict):
                prompt = next(iter(prompt.values()))
            else:
                prompt = str(prompt)
        resp = client.chat.completions.create(
            model = "gpt-4o",
            messages = [{"role": "user", "content": prompt}]
        )
        text = resp.choices[0].message.content
        return re.sub("\u200c", "", text)

In [5]:
llm = SimpleLLM()

In [31]:
tavily_search = TavilySearchResults(max_results = 1, tavily_api_key = "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")
wiki_wrapper = WikipediaAPIWrapper(lang = "fa", top_k_results = 1)
wiki_tool = WikipediaQueryRun(api_wrapper = wiki_wrapper)
youtube_tool = YouTubeSearchTool(description = "جستجوی ویدیوهای مرتبط در YouTube. ورودی باید به صورت '<موضوع>، <تعداد>' باشد.")
llm_math = LLMMathChain.from_llm(llm = llm, verbose = True)
calc_tool = Tool(
    name = "Calculator",
    func = llm_math.run,
    description = "برای محاسبات عددی و محاسباتی از این ابزار استفاده کن."
)

In [32]:
tools = [tavily_search, wiki_tool, youtube_tool, calc_tool]

In [35]:
agent = initialize_agent(
    tools, 
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True,
    handle_parsing_errors = True
)

queries = ["آخرین اخبار درباره مذاکرات ایران با سه کشور اروپایی",
          "علی دایی کجا به دنیا آمده است و در سال 2025 چند سال سن دارد؟",
          "یک شعر درباره دریا بنویس.",
          "در یوتیوب 2 ویدیو درباره آموزش پخت کیک شکلاتی پیدا کن."]
for query in queries:
    response = agent.run(query)
    print(response)
    print("#" * 80)



> Entering new AgentExecutor chain...
برای پاسخ به این سوال، نیاز است تا اخبار بهروز و مرتبط درباره مذاکرات ایران با سه کشور اروپایی را جستجو کنم.

Action: tavily_search_results_json  
Action Input: "آخرین اخبار درباره مذاکرات ایران با سه کشور اروپایی"  
Observation: [{'title': 'مذاکرات هسته\u200cای ایران و سه کشور اروپایی در استانبول آغاز شد - BBC', 'url': 'https://www.bbc.com/persian/articles/c15w5xv5qxyo', 'content': '# آغاز مذاکرات هسته\u200cای ایران و سه کشور اروپایی در استانبول؛ تهران خواستار «جبران رویکردهای غیرسازنده» شد\n\nمذاکرات در کنسولگری جمهوری اسلامی ایران در استانبول برگزار می\u200cشود\n\nمنبع تصویر، Reuters\n\nمذاکرات هسته ای ایران و سه کشور اروپایی بریتانیا، فرانسه و آلمان در استانبول آغاز شده است. این گفت\u200cوگوها در سطح معاونان وزارت خارجه چهار کشور و در کنسولگری جمهوری اسلامی ایران در استانبول برگزار می\u200cشود. [...] پس از جگ ۱۲ روزه اسرائیل و ایران و بعد از از ضرب\u200cالاجل اروپا برای رسیدن به یک توافق هسته\u200cای جدید، این اولین مذاکراتی است که با ایران ا